# Selecting Master Sample for Virgo Filament Survey #

### Post ISSI 2019 ###

- Gianluca has made a master sample from Hyperleda
- In Bern, we were trying to reproduce the steps he took to:
    - cross-match with NSA, and 
    - remove spurious detections
- Gianluca matched according to NED name.
- RF is going to do the same thing but matching according to RA and DEC.